In [1]:
from typing import Dict, Optional, List
import datetime as dt
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.contrib.latex_printer import latex_printer

In [2]:
def build_model(prices: List[int], objective_limit: float = float('inf')) -> pyo.ConcreteModel:
    mdl = pyo.ConcreteModel()

    mdl.lot = pyo.Set(initialize=[1, 2])
    mdl.scenarios = pyo.Set(initialize=[f"s{i}" for i in range(len(prices))])

    mdl.land_development_cost = pyo.Param(mdl.lot, initialize={1: 600, 2: 100})
    mdl.build_cost_now = pyo.Param(mdl.lot, initialize={1: 200, 2: 600})
    mdl.build_cost_later = pyo.Param(mdl.lot, initialize={1: 220, 2: 660})
    mdl.price_scenarios = pyo.Param(mdl.scenarios, initialize={f"s{i}": price for i, price in enumerate(prices)})

    mdl.develop_land = pyo.Var(mdl.lot, domain=pyo.Binary, name="Develop land")
    mdl.build_now = pyo.Var(mdl.lot, domain=pyo.Binary, name="Build now")
    mdl.build_later = pyo.Var(mdl.scenarios, mdl.lot, domain=pyo.Binary, name="Build later")

    mdl.objective = pyo.Objective(
        expr=(
            sum(
                1/len(mdl.scenarios) * sum(
                    mdl.price_scenarios[sc] * (mdl.build_now[lot] + mdl.build_later[sc, lot])
                    - mdl.build_cost_later[lot] * mdl.build_later[sc, lot]
                    for lot in mdl.lot
                )
                for sc in mdl.scenarios
            )
            - sum(
                mdl.build_cost_now[lot] * mdl.build_now[lot]
                for lot in mdl.lot
            )
            - sum(
                mdl.land_development_cost[lot] * mdl.develop_land[lot]
                for lot in mdl.lot
            )
        ),
        sense=pyo.maximize,
    )

    mdl.constr_build_once = pyo.ConstraintList()
    for sc in mdl.scenarios:
        for lot in mdl.lot:
            mdl.constr_build_once.add(mdl.build_now[lot] + mdl.build_later[sc, lot] <= 1)

    mdl.constr_cost_of_land = pyo.ConstraintList()
    for sc in mdl.scenarios:
        for lot in mdl.lot:
            mdl.constr_cost_of_land.add(mdl.develop_land[lot] >= (mdl.build_now[lot] + mdl.build_later[sc, lot]))

    mdl.limit_optimal_solution = pyo.Constraint(
        expr=mdl.objective <= objective_limit
    )

    return mdl

In [3]:
prices = [210, 1250, 2500]
mdl = build_model(prices=prices)
solver = pyo.SolverFactory('scip')
results = solver.solve(mdl)

In [4]:
objective = mdl.objective()
build_now = mdl.build_now.extract_values()
build_later = mdl.build_later.extract_values()
develop_land = mdl.develop_land.extract_values()

print("Build now: ", build_now)
print("Build later: ", build_later)
print("Develop land: ", develop_land)
print("Objective value: ", objective)

Build now:  {1: 1.0, 2: -0.0}
Build later:  {('s0', 1): -0.0, ('s0', 2): -0.0, ('s1', 1): 0.0, ('s1', 2): 1.0, ('s2', 1): 0.0, ('s2', 2): 1.0}
Develop land:  {1: 1.0, 2: 1.0}
Objective value:  1230.0


### Custom solutions

In [6]:
build_now = {1: 0, 2: 0}
build_later = {('s1', 1): 0, ('s1', 2): 0, ('s2', 1): 0, ('s2', 2): 1}
develop_land = {1: 0, 2: 1}

In [7]:
(
    sum(
        1/len(mdl.scenarios) * sum(
            mdl.price_scenarios[sc] * ((build_now[lot] + build_later[sc, lot]))
            - mdl.build_cost_later[lot] * build_later[sc, lot]
            for lot in mdl.lot
        )
        for sc in mdl.scenarios
    )
    - sum(
        mdl.build_cost_now[lot] * build_now[lot]
        for lot in mdl.lot
    )
    - sum(
        mdl.land_development_cost[lot] * develop_land[lot]
        for lot in mdl.lot
    )
)

195.0

In [8]:
sum(
    1/len(mdl.scenarios) * sum(
        mdl.price_scenarios[sc] * (build_now[lot] + build_later[sc, lot])
        - mdl.build_cost_later[lot] * build_later[sc, lot]
        for lot in mdl.lot
    )
    for sc in mdl.scenarios
)

295.0

In [9]:
(
    - sum(
        mdl.build_cost_now[lot] * build_now[lot]
        for lot in mdl.lot
    )
    - sum(
        mdl.land_development_cost[lot] * develop_land[lot]
        for lot in mdl.lot
    )
)

-100